In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,ElasticNet,Lasso,Ridge
from sklearn.linear_model import SGDRegressor


In [2]:
df_train=pd.read_csv("house_price_train.csv")
df_train.drop("Id",axis=1,inplace=True) ## reduntatnt 
output_col=df_train['SalePrice']
df_train=df_train.drop('SalePrice',axis=1)

In [3]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal


In [4]:
df_train.dtypes

MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
                  ...   
MiscVal            int64
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
Length: 79, dtype: object

## PREPROCESSING

In [5]:
df_train.columns[df_train.isnull().any()]

Index(['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence',
       'MiscFeature'],
      dtype='object')

In [6]:
# sns.heatmap(df_train.isnull(),yticklabels=False,cbar=False,cmap='viridis')

In [7]:
# plt.figure(figsize=(30,30))
# sns.heatmap(pd.concat([df_train,output_col],axis=1).loc[:,list(pd.concat([df_train,output_col],axis=1).select_dtypes(include= np.number).columns)].corr(method='spearman'),annot=True,cmap="RdYlGn")
# plt.show()

In [8]:
# fig, axes = plt.subplots(nrows = 12, ncols = 3)    # axes is 2d array (3x3)
# axes = axes.flatten()         # Convert axes to 1d array of length 9
# fig.set_size_inches(30,30)
# cols=list(df_train.select_dtypes(include= np.number).columns)
# for ax, col in zip(axes, cols):
#     print()
#     sns.boxplot(df_train[col], ax = ax)
#     ax.set_title(col)

In [9]:
# fig, axes = plt.subplots(nrows = 12, ncols = 3)    # axes is 2d array (3x3)
# axes = axes.flatten()         # Convert axes to 1d array of length 9
# fig.set_size_inches(30,30)
# cols=list(df_train.select_dtypes(include= np.number).columns)
# for ax, col in zip(axes, cols):
#     print()
#     sns.distplot(df_train[col], ax = ax)
#     ax.set_title(col)

In [10]:
def column_with_most_reduntant_null_values(threshold,df_train:pd.DataFrame):
    null_cols=[]
    for col in df_train.columns:
        if df_train[col].isnull().sum()>=(threshold*len(df_train)):
            null_cols.append(col)
    df_train.drop(null_cols,axis=1,inplace=True)

In [11]:
column_with_most_reduntant_null_values(threshold=0.5,df_train=df_train)

In [12]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,61,0,0,0,0,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,42,0,0,0,0,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,35,272,0,0,0,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,84,0,0,0,0,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,40,0,0,0,0,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,60,0,0,0,0,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,112,0,0,0,0,4,2010,WD,Normal


In [13]:
continuous_cols=list(df_train.select_dtypes(include= np.number).columns)

In [14]:
categorical_cols=list(df_train.select_dtypes(exclude= np.number).columns)

In [15]:
most_correlated=dict()
for col1 in continuous_cols:
    for col2 in continuous_cols:
        if col1!=col2 and abs(df_train[col1].corr(df_train[col2],method='spearman'))>=0.80:
            if (col1 not in most_correlated) and (col2 not in most_correlated):
                most_correlated[col1]=col2

In [16]:
df_train.drop(columns=list(most_correlated),axis=1,inplace=True)

In [17]:
def fill_null_values(for_numerical_cols:str,for_categorical_cols:str,df_train):
    for cols in df_train.columns:
        if df_train[cols].dtype=="int" or df_train[cols].dtype=="float":
            if for_numerical_cols=="median":
                median_=df_train[cols].median()
                df_train[cols].fillna(median_,inplace=True)
            elif for_numerical_cols=="mode":
                mode_=df_train[cols].mode()[0]
                df_train[cols].fillna(mode_,inplace=True)
        elif df_train[cols].dtype=="object":
            if for_categorical_cols=="most_frequent":
                d=list(df_train[cols].value_counts().index) # most frequent
                df_train[cols].fillna(d[0],inplace=True)

In [18]:
fill_null_values(for_numerical_cols="median",for_categorical_cols="most_frequent",df_train=df_train)

In [19]:
for col in df_train.columns:
    if df_train[col].isnull().sum()>(0):

        print(col,df[col].isnull().sum(),df_train[col].dtypes)


In [20]:
continuous_cols=list(df_train.select_dtypes(include= np.number).columns)

In [21]:
categorical_cols=list(df_train.select_dtypes(exclude= np.number).columns)

Deleting Outliers.

Here we are deleting outliers using Inter Quartile Range Strategy. Since for a column ouliers are in a specifc row and for different columns the oulier contained rows are different and if we delete those specific row then that row in output label should also be deleted thats why we are first adding label into our features after that we will separate the label.

In [22]:
df_train=pd.concat([output_col,df_train],axis=1) # since rows will be deleted here

In [23]:
for col in continuous_cols:
    q1=df_train[col].quantile(0.25)
    q3=df_train[col].quantile(0.75)
    iqr=q3-q1
    l=q1-1.5*iqr
    h=q3+1.5*iqr
    print("[",l,",",h,"]")
    df_train = df_train[(df_train[col] <= h)] 
    df_train = df_train[(df_train[col] >=l)] 



[ -55.0 , 145.0 ]
[ 30.0 , 110.0 ]
[ 2440.0 , 16936.0 ]
[ 2.0 , 10.0 ]
[ 3.5 , 7.5 ]
[ 1909.0 , 2061.0 ]
[ -255.75 , 426.25 ]
[ -1060.5 , 1767.5 ]
[ 0.0 , 0.0 ]
[ -579.875 , 1725.125 ]
[ 199.0 , 2037.0 ]
[ -1092.375 , 1820.625 ]
[ 0.0 , 0.0 ]
[ -1.5 , 2.5 ]
[ 0.0 , 0.0 ]
[ -0.5 , 3.5 ]
[ -1.5 , 2.5 ]
[ 0.5 , 4.5 ]
[ 1.0 , 1.0 ]
[ 2.0 , 10.0 ]
[ -1.5 , 2.5 ]
[ 1903.0 , 2063.0 ]
[ -19.0 , 933.0 ]
[ -239.625 , 399.375 ]
[ -97.5 , 162.5 ]
[ 0.0 , 0.0 ]
[ 0.0 , 0.0 ]
[ 0.0 , 0.0 ]
[ 0.0 , 0.0 ]
[ 0.0 , 0.0 ]
[ 0.5 , 12.5 ]
[ 2004.0 , 2012.0 ]


In [24]:
output_col=df_train['SalePrice']
df_train.drop("SalePrice",axis=1,inplace=True)

In [25]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,61,0,0,0,0,0,2,2008,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,42,0,0,0,0,0,9,2008,WD,Normal
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,84,0,0,0,0,0,12,2008,WD,Normal
6,20,RL,75.0,10084,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,57,0,0,0,0,0,8,2007,WD,Normal
10,20,RL,70.0,11200,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,2,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444,20,RL,63.0,8500,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,60,0,0,0,0,0,11,2007,WD,Normal
1448,50,RL,70.0,11767,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,24,0,0,0,0,0,5,2007,WD,Normal
1451,20,RL,78.0,9262,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,36,0,0,0,0,0,5,2009,New,Partial
1454,20,FV,62.0,7500,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,113,0,0,0,0,0,10,2009,WD,Normal


Here we are encoding our categorical columns using 'one_hot_encoding' as a strategy. We will be experimenting other strategies later.

In [26]:

def encoding_categorical_cols(df_train:pd.DataFrame,categorical_columns:list,strategy:str):
    df_categorical=pd.DataFrame(df_train.loc[:,categorical_columns])
    if strategy=="labelencoder":
        for col in list(df_categorical.columns):
            cat=[]
            d={}

            cat.append([df_categorical[col].value_counts().index,len(df_categorical[col].value_counts().index)])

            for j in range(int(cat[0][1])):
                d[str(cat[0][0][j])]=j
            df_categorical[col]=df_categorical[col].map(d)
        df_train.drop(categorical_columns,axis=1,inplace=True)
        
        df_train=pd.concat([df_train,df_categorical],axis=1)
        
    if strategy=="onehotencoder":
        one_hot_encoded=pd.get_dummies(df_categorical)
        # print(one_hot_encoded)
        df_train.drop(categorical_columns,axis=1,inplace=True)
        df_train=pd.concat([df_train,one_hot_encoded],axis=1)
    return df_train

In [27]:
df_train=encoding_categorical_cols(df_train=df_train,categorical_columns=categorical_cols,strategy="onehotencoder")

In [28]:
df_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,196.0,706,0,150,...,0,0,0,1,0,0,0,0,1,0
2,60,68.0,11250,7,5,2002,162.0,486,0,434,...,0,0,0,1,0,0,0,0,1,0
4,60,84.0,14260,8,5,2000,350.0,655,0,490,...,0,0,0,1,0,0,0,0,1,0
6,20,75.0,10084,8,5,2005,186.0,1369,0,317,...,0,0,0,1,0,0,0,0,1,0
10,20,70.0,11200,5,5,1965,0.0,906,0,134,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444,20,63.0,8500,7,5,2004,106.0,0,0,1422,...,0,0,0,1,0,0,0,0,1,0
1448,50,70.0,11767,4,7,2000,0.0,0,0,560,...,0,0,0,1,0,0,0,0,1,0
1451,20,78.0,9262,8,5,2009,194.0,0,0,1573,...,0,1,0,0,0,0,0,0,0,1
1454,20,62.0,7500,7,5,2005,0.0,410,0,811,...,0,0,0,1,0,0,0,0,1,0


In [29]:
df_train.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
count,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.0,576.000000,...,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000
mean,48.315972,68.781250,9167.371528,6.263889,5.366319,1989.269097,77.281250,441.932292,0.0,614.366319,...,0.001736,0.119792,0.003472,0.835069,0.060764,0.001736,0.001736,0.012153,0.802083,0.121528
std,31.355368,13.663944,2609.625982,1.316047,0.704898,19.823626,108.909316,416.412209,0.0,428.094824,...,0.041667,0.325000,0.058874,0.371440,0.239104,0.041667,0.041667,0.109663,0.398775,0.327024
min,20.000000,30.000000,2887.000000,2.000000,4.000000,1950.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,20.000000,60.000000,7717.250000,5.000000,5.000000,1972.000000,0.000000,0.000000,0.0,280.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,50.000000,69.000000,9017.500000,6.000000,5.000000,1999.000000,0.000000,428.000000,0.0,517.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,60.000000,75.000000,10800.000000,7.000000,6.000000,2005.000000,147.250000,743.000000,0.0,879.250000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,120.000000,110.000000,16770.000000,10.000000,7.000000,2010.000000,423.000000,1619.000000,0.0,1709.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Taking features in X and lables in Y

In [30]:
X=df_train
Y=output_col

The below dictionary will save the final accuracies of each model.

In [35]:
avg_accuracies={}
highest_accuracies={}
algorithms_tested=["LinearRegression()","RandomForestRegressor()","SVR()","KNeighborsRegressor()","SGDRegressor()","DecisionTreeRegressor()","ElasticNet()","Lasso()","Ridge()"]
for al in algorithms_tested:
    avg_accuracies[al]=0
    highest_accuracies[al]=float('-inf')

In [36]:
def algo_testing(algo:str,test_size:float,X,Y,evaluation_metric:str,random_state,shuffle):
    model=eval(algo)
    evaluation_metric=eval(evaluation_metric)
    X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=random_state,shuffle=shuffle)
   
    model.fit(X_train,Y_train)
    predicted=model.predict(X_test)
    
    r2=evaluation_metric(Y_test, predicted)
    print(f"Testing r2_score of {model} is ",r2)
    print(f"Training r2_score of {model} is ", model.score(X_train,Y_train))
    if highest_accuracies[algo]<r2:
        
        highest_accuracies[algo]=r2
    avg_accuracies[algo]+=r2



In [37]:
for i in range(50):
    algo_testing(algo="LinearRegression()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["LinearRegression()"]/=50

Testing r2_score of LinearRegression() is  0.9076775940055598
Training r2_score of LinearRegression() is  0.9605699283243948
Testing r2_score of LinearRegression() is  0.8737982126333906
Training r2_score of LinearRegression() is  0.9642734927817916
Testing r2_score of LinearRegression() is  0.8858657662394811
Training r2_score of LinearRegression() is  0.9645667065992024
Testing r2_score of LinearRegression() is  0.8398725786115062
Training r2_score of LinearRegression() is  0.9660555205508169
Testing r2_score of LinearRegression() is  0.896758151502679
Training r2_score of LinearRegression() is  0.9624052613253443
Testing r2_score of LinearRegression() is  0.8695118332607018
Training r2_score of LinearRegression() is  0.9623843560152299
Testing r2_score of LinearRegression() is  0.8583229743530547
Training r2_score of LinearRegression() is  0.9646832986447157
Testing r2_score of LinearRegression() is  0.9040456851563488
Training r2_score of LinearRegression() is  0.962356497821478
Te

In [38]:
for i in range(50):

    algo_testing(algo="RandomForestRegressor()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["RandomForestRegressor()"]/=50

Testing r2_score of RandomForestRegressor() is  0.8578257006228268
Training r2_score of RandomForestRegressor() is  0.9838527563675534
Testing r2_score of RandomForestRegressor() is  0.8690999474500988
Training r2_score of RandomForestRegressor() is  0.9807544489346973
Testing r2_score of RandomForestRegressor() is  0.8904146078783167
Training r2_score of RandomForestRegressor() is  0.9816281755618614
Testing r2_score of RandomForestRegressor() is  0.9021462727139775
Training r2_score of RandomForestRegressor() is  0.9822288755250917
Testing r2_score of RandomForestRegressor() is  0.8770262396297277
Training r2_score of RandomForestRegressor() is  0.9808265455215197
Testing r2_score of RandomForestRegressor() is  0.8676221986323024
Training r2_score of RandomForestRegressor() is  0.9835879432135222
Testing r2_score of RandomForestRegressor() is  0.8293706216793074
Training r2_score of RandomForestRegressor() is  0.982684910550081
Testing r2_score of RandomForestRegressor() is  0.899656

In [39]:
for i in range(50):

    algo_testing(algo="SVR()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["SVR()"]/=50

Testing r2_score of SVR() is  -0.004289540254449831
Training r2_score of SVR() is  -0.010218311717025363
Testing r2_score of SVR() is  -0.06685916894468558
Training r2_score of SVR() is  -0.007692451633748876
Testing r2_score of SVR() is  -0.04158864258378059
Training r2_score of SVR() is  -0.011942934599658894
Testing r2_score of SVR() is  -0.001302483324164605
Training r2_score of SVR() is  -0.013784912591914278
Testing r2_score of SVR() is  -0.03847742611684679
Training r2_score of SVR() is  -0.012044722154397558
Testing r2_score of SVR() is  -0.006217907126961908
Training r2_score of SVR() is  -0.009608304303465598
Testing r2_score of SVR() is  -0.03661416144664065
Training r2_score of SVR() is  -0.0088321386257455
Testing r2_score of SVR() is  -0.03313896746975131
Training r2_score of SVR() is  -0.005046948158841547
Testing r2_score of SVR() is  -0.001045517577940025
Training r2_score of SVR() is  -0.009980462743701457
Testing r2_score of SVR() is  -6.949747989981603e-05
Training 

In [40]:
for i in range(50):

    algo_testing(algo="KNeighborsRegressor()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["KNeighborsRegressor()"]/=50

Testing r2_score of KNeighborsRegressor() is  0.7140539352780529
Training r2_score of KNeighborsRegressor() is  0.7649551681090667
Testing r2_score of KNeighborsRegressor() is  0.6447155641576279
Training r2_score of KNeighborsRegressor() is  0.7737546516122604
Testing r2_score of KNeighborsRegressor() is  0.6894510792642335
Training r2_score of KNeighborsRegressor() is  0.7471159537696828
Testing r2_score of KNeighborsRegressor() is  0.5759256264598681
Training r2_score of KNeighborsRegressor() is  0.8060420758812686
Testing r2_score of KNeighborsRegressor() is  0.6958028836023655
Training r2_score of KNeighborsRegressor() is  0.7692709311547101
Testing r2_score of KNeighborsRegressor() is  0.7058792233076987
Training r2_score of KNeighborsRegressor() is  0.7462316213133439
Testing r2_score of KNeighborsRegressor() is  0.6942565474955096
Training r2_score of KNeighborsRegressor() is  0.7643023419442125
Testing r2_score of KNeighborsRegressor() is  0.7015136392131875
Training r2_score 

In [41]:
for i in range(50):

    algo_testing(algo="SGDRegressor()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["SGDRegressor()"]/=50

Testing r2_score of SGDRegressor() is  -1.4084619123382488e+23
Training r2_score of SGDRegressor() is  -1.2025649909267605e+23
Testing r2_score of SGDRegressor() is  -5.7282382126264875e+22
Training r2_score of SGDRegressor() is  -5.121815908364412e+22
Testing r2_score of SGDRegressor() is  -6.746250335601562e+22
Training r2_score of SGDRegressor() is  -7.229935352006e+22
Testing r2_score of SGDRegressor() is  -2.003795324152878e+23
Training r2_score of SGDRegressor() is  -1.5374704696869598e+23
Testing r2_score of SGDRegressor() is  -8.053203268404233e+22
Training r2_score of SGDRegressor() is  -9.124925448417488e+22
Testing r2_score of SGDRegressor() is  -1.341003379067694e+24
Training r2_score of SGDRegressor() is  -1.4475817215738355e+24
Testing r2_score of SGDRegressor() is  -7.72231936950965e+22
Training r2_score of SGDRegressor() is  -4.9093692447063145e+22
Testing r2_score of SGDRegressor() is  -1.4347276784370016e+23
Training r2_score of SGDRegressor() is  -1.4070934789223863e

In [42]:
for i in range(50):

    algo_testing(algo="DecisionTreeRegressor()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["DecisionTreeRegressor()"]/=50

Testing r2_score of DecisionTreeRegressor() is  0.6831155424651253
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.5976036529290842
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.7271670123266134
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.7379744522334939
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.6842259184003157
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.5707381194910901
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.6883404455597415
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.7628813506951265
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegresso

In [43]:
for i in range(50):

    algo_testing(algo="ElasticNet()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["ElasticNet()"]/=50

Testing r2_score of ElasticNet() is  0.8935788916452637
Training r2_score of ElasticNet() is  0.8970737420784367
Testing r2_score of ElasticNet() is  0.8088757095616577
Training r2_score of ElasticNet() is  0.9152231597515018
Testing r2_score of ElasticNet() is  0.9175588510274056
Training r2_score of ElasticNet() is  0.8964487239702479
Testing r2_score of ElasticNet() is  0.8350900870613429
Training r2_score of ElasticNet() is  0.9090546067219969
Testing r2_score of ElasticNet() is  0.822183993707116
Training r2_score of ElasticNet() is  0.9115726965820945
Testing r2_score of ElasticNet() is  0.849521492349538
Training r2_score of ElasticNet() is  0.9070688361841773
Testing r2_score of ElasticNet() is  0.8770661297498032
Training r2_score of ElasticNet() is  0.903976361012362
Testing r2_score of ElasticNet() is  0.887711830561843
Training r2_score of ElasticNet() is  0.899630680545798
Testing r2_score of ElasticNet() is  0.9076705535326699
Training r2_score of ElasticNet() is  0.89556

In [44]:
for i in range(50):

    algo_testing(algo="Lasso()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["Lasso()"]/=50

C:\Users\mritu\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.518e+10, tolerance: 1.929e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\mritu\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.453e+10, tolerance: 1.927e+08
  model = cd_fast.enet_coordinate_descent(


Testing r2_score of Lasso() is  0.9016652399509895
Training r2_score of Lasso() is  0.9599294302742994
Testing r2_score of Lasso() is  0.8461512222140986
Training r2_score of Lasso() is  0.9648985134094196


C:\Users\mritu\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.634e+10, tolerance: 1.924e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\mritu\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.179e+10, tolerance: 1.872e+08
  model = cd_fast.enet_coordinate_descent(


Testing r2_score of Lasso() is  0.8704778892647205
Training r2_score of Lasso() is  0.9629039901838862
Testing r2_score of Lasso() is  0.8974772785414198
Training r2_score of Lasso() is  0.9594906401993455


C:\Users\mritu\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.863e+10, tolerance: 1.899e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\mritu\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.032e+10, tolerance: 1.719e+08
  model = cd_fast.enet_coordinate_descent(


...

Testing r2_score of Lasso() is  0.9139042638972993
Training r2_score of Lasso() is  0.9595822266448439
Testing r2_score of Lasso() is  0.9210107070409673
Training r2_score of Lasso() is  0.9586901165128878


C:\Users\mritu\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.851e+10, tolerance: 1.878e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\mritu\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.931e+10, tolerance: 1.694e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\mritu\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

Testing r2_score of Lasso() is  0.8970936302982718
Training r2_score of Lasso() is  0.9640555786484877
Testing r2_score of Lasso() is  0.8907003493835173
Training r2_score of Lasso() is  0.9611396150942672
Testing r2_score of Lasso() is  0.8862460422367241


C:\Users\mritu\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.503e+10, tolerance: 1.863e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\mritu\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.388e+10, tolerance: 1.835e+08
  model = cd_fast.enet_coordinate_descent(


Training r2_score of Lasso() is  0.9625001680486277
Testing r2_score of Lasso() is  0.9068330285240234
Training r2_score of Lasso() is  0.9623029389402469


In [45]:
for i in range(50):

    algo_testing(algo="Ridge()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["Ridge()"]/=50

Testing r2_score of Ridge() is  0.8890862024903524
Training r2_score of Ridge() is  0.9598516836223839
Testing r2_score of Ridge() is  0.884923734025131
Training r2_score of Ridge() is  0.963417630569889
Testing r2_score of Ridge() is  0.9128823059036297
Training r2_score of Ridge() is  0.9561524787450203
Testing r2_score of Ridge() is  0.9227739084638149
Training r2_score of Ridge() is  0.9552419741250476
Testing r2_score of Ridge() is  0.9065329281020633
Training r2_score of Ridge() is  0.9580245339694632
Testing r2_score of Ridge() is  0.9308784079814445
Training r2_score of Ridge() is  0.9547806055291573
Testing r2_score of Ridge() is  0.9189950486676709
Training r2_score of Ridge() is  0.95566025370157
Testing r2_score of Ridge() is  0.9008313399065957
Training r2_score of Ridge() is  0.960811338904965
Testing r2_score of Ridge() is  0.9282540824276736
Training r2_score of Ridge() is  0.9560886345098776
Testing r2_score of Ridge() is  0.8677321280054626
Training r2_score of Ridge(

In [46]:
avg_accuracies

{'LinearRegression()': 0.881031284342253,
 'RandomForestRegressor()': 0.869510656363165,
 'SVR()': -0.01728274533459442,
 'KNeighborsRegressor()': 0.6421184797862709,
 'SGDRegressor()': -3.216932676348004e+23,
 'DecisionTreeRegressor()': 0.7185941932065119,
 'ElasticNet()': 0.8806240886157383,
 'Lasso()': 0.8874982341701191,
 'Ridge()': 0.905736300566432}

In [47]:
highest_accuracies

{'LinearRegression()': 0.9278445170816145,
 'RandomForestRegressor()': 0.930696214382794,
 'SVR()': 0.0001507364448004589,
 'KNeighborsRegressor()': 0.7815599963584625,
 'SGDRegressor()': -2.8317133136364034e+21,
 'DecisionTreeRegressor()': 0.8103796408179733,
 'ElasticNet()': 0.9404476560657553,
 'Lasso()': 0.9320448373151158,
 'Ridge()': 0.9364811978664624}